In [1]:
import math
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import collections as col
import re
import random
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
#from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier

In [2]:
def load_data(train='Yes', test='No', validation='No'):
	"""Loads and returns datasets as required
	   Return empty lst for if 'No'
	"""
	if train=='Yes':
		df_train = pd.read_csv('dataset/train.csv', sep=',')
	else:
		df_train = []

	if test=='Yes':
		df_test = pd.read_csv('dataset/test.csv', sep=',')
	else:
		df_test = []

	if validation=='Yes':
		df_validation = pd.read_csv('dataset/validation.csv', sep=',')
	else:
		df_validation = []
	
	print('Data loaded', len(df_train), len(df_test), len(df_validation))
	return df_train, df_test, df_validation

In [3]:
df_train, df_test, df_validation= load_data('Yes', 'No', 'Yes')

Data loaded 2697738 0 299749


## pCTR, Logistic regression and Naive Bayes

In [6]:
# Feature engineering
def label_encoder(df_column_nint, column_le= None): 
    if column_le== None:
        column_le = LabelEncoder()
        column_le.fit(df_column_nint.unique())

    df_column_int = column_le.transform(df_column_nint)
    return pd.DataFrame(df_column_int), column_le

def onehot_encoder(df_column_nohe, OHE= None): 
    
    ar = df_column_nohe.reshape((-1, 1))
    ar_u = np.unique(ar)
    ar_u_r = ar_u.reshape((-1, 1))
    ar_u_r.shape
    
    if OHE== None:
        OHE= OneHotEncoder()
        OHE.fit(ar_u_r)
    
    return pd.DataFrame(OHE.transform(ar).toarray()), OHE

In [7]:
df_LE_train = pd.DataFrame()
df_LE_train = df_train

# Y - Target
array_y = df_LE_train[['click']].as_matrix()
array_y_r = np.reshape(array_y, (-1))

# X - Features
# Label encoding - NB
le_adexchange, le_a1 = label_encoder(df_LE_train['adexchange'])
le_slotformat, le_s2 = label_encoder(df_LE_train['slotformat'])
le_slotvisibility, le_s3 = label_encoder(df_LE_train['slotvisibility'])
le_useragent, le_u4 = label_encoder(df_LE_train['useragent'])

df_LE_train['le_adexchange']= le_adexchange
df_LE_train['le_slotformat']= le_slotformat
df_LE_train['le_slotvisibility']= le_slotvisibility
df_LE_train['le_useragent']= le_useragent

In [8]:
df_train.columns

Index(['click', 'weekday', 'hour', 'bidid', 'logtype', 'userid', 'useragent',
       'IP', 'region', 'city', 'adexchange', 'domain', 'url', 'urlid',
       'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat',
       'slotprice', 'creative', 'bidprice', 'payprice', 'keypage',
       'advertiser', 'usertag', 'le_adexchange', 'le_slotformat',
       'le_slotvisibility', 'le_useragent'],
      dtype='object')

In [9]:
df_OHE_train = pd.DataFrame()
df_OHE_train = df_train

ohe_weekday, ohe_w1= onehot_encoder(df_OHE_train['weekday'].values)
ohe_hour, ohe_h2= onehot_encoder(df_OHE_train['hour'].values)
ohe_slotwidth, ohe_s3= onehot_encoder(df_OHE_train['slotwidth'].values)
ohe_slotheight, ohe_s4= onehot_encoder(df_OHE_train['slotheight'].values)
ohe_advertiser, ohe_a5= onehot_encoder(df_OHE_train['advertiser'].values)
ohe_city, ohe_c6= onehot_encoder(df_OHE_train['city'].values)

# Requires LE columns also from df_LE_train
ohe_adexchange, ohe_a7= onehot_encoder(df_LE_train['le_adexchange'].values)
ohe_slotformat, ohe_s8= onehot_encoder(df_LE_train['le_slotformat'].values)
ohe_slotvisibility, ohe_s9= onehot_encoder(df_LE_train['le_slotvisibility'].values)
ohe_useragent, ohe_u10= onehot_encoder(df_LE_train['le_useragent'].values)

In [10]:
df_OHE_train_feat= pd.DataFrame()
df_OHE_train_feat= pd.concat([ohe_weekday, ohe_hour, ohe_slotwidth, ohe_slotheight, ohe_advertiser, ohe_city, \
                             ohe_adexchange, ohe_slotformat, ohe_slotvisibility, ohe_useragent], axis=1, join='inner')
df_OHE_train_feat.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  0,  1,  2,
            ...
            29, 30, 31, 32, 33, 34, 35, 36, 37, 38],
           dtype='int64', length=504)

In [11]:
LR_OHE_array_x_all= df_OHE_train_feat.as_matrix()

In [ ]:
lst_x_1 = ['advertiser']
lst_x_2 = lst_int_features
lst_x_3 = [lst_int_features[0], lst_int_features[1], lst_nint_le_features[0]]
lst_x_all = lst_int_features+ lst_nint_le_features

LE_array_x_1 = df_LE_train[lst_x_1].as_matrix()
LE_array_x_2 = df_LE_train[lst_x_2].as_matrix()
LE_array_x_3 = df_LE_train[lst_x_3].as_matrix()
LE_array_x_all = df_LE_train[lst_x_all].as_matrix()

In [ ]:
#OHE_array_x_1 = df_OHE_train[lst_x_1].as_matrix()
#OHE_array_x_2 = df_OHE_train[lst_x_2].as_matrix()
#OHE_array_x_3 = df_OHE_train[lst_x_3].as_matrix()
OHE_array_x_all = df_OHE_train.iloc[35:539].as_matrix()

In [13]:
# Build model
def Naive_Bayes(array_x, array_y):
    NB_model = GaussianNB()
    NB_model.fit(array_x, array_y)
    return NB_model

def Logistic_Regression(array_x, array_y):
    LR_model = linear_model.LogisticRegression(penalty='l2')
    LR_model.fit(array_x, array_y)
    return LR_model

def AdaBoost_Classifier(array_x, array_y):
    AB_model = AdaBoostClassifier()
    AB_model.fit(array_x, array_y)
    return AB_model

# Build with on 'advertiser' from validation set
#NB_model_1 = Naive_Bayes(array_x_1, array_y_r)
#NB_model_2 = Naive_Bayes(array_x_2, array_y_r)
#NB_model_3 = Naive_Bayes(array_x_3, array_y_r)
#NB_model_all = Naive_Bayes(array_x_all, array_y_r)

#LR_model_1 = Logistic_Regression(array_x_1, array_y_r)
#LR_model_2 = Logistic_Regression(array_x_2, array_y_r)
#LR_model_3 = Logistic_Regression(array_x_3, array_y_r)
#LR_model_all = Logistic_Regression(array_x_all, array_y_r)

AB_model_all = AdaBoost_Classifier(LR_OHE_array_x_all, array_y_r)

In [12]:
# Pipeline validation - Take train ohe_x# and le_x#, to convert validation 
ohe_weekday_val, _= onehot_encoder(df_validation['weekday'].values, ohe_w1)
ohe_hour_val, _= onehot_encoder(df_validation['hour'].values, ohe_h2)
ohe_slotwidth_val, _= onehot_encoder(df_validation['slotwidth'].values, ohe_s3)
ohe_slotheight_val, _= onehot_encoder(df_validation['slotheight'].values, ohe_s4)
ohe_advertiser_val, _= onehot_encoder(df_validation['advertiser'].values, ohe_a5)
ohe_city_val, _= onehot_encoder(df_validation['city'].values, ohe_c6)

le_adexchange_val, _ = label_encoder(df_validation['adexchange'], le_a1)
le_slotformat_val, _ = label_encoder(df_validation['slotformat'], le_s2)
le_slotvisibility_val, _ = label_encoder(df_validation['slotvisibility'], le_s3)
le_useragent_val, _ = label_encoder(df_validation['useragent'], le_u4)

ohe_adexchange_val, _= onehot_encoder(le_adexchange_val.values, ohe_a7)
ohe_slotformat_val, _= onehot_encoder(le_slotformat_val.values, ohe_s8)
ohe_slotvisibility_val, _= onehot_encoder(le_slotvisibility_val.values, ohe_s9)
ohe_useragent_val, _= onehot_encoder(le_useragent_val.values, ohe_u10)

df_OHE_val_feat= pd.DataFrame()
df_OHE_val_feat= pd.concat([ohe_weekday_val, ohe_hour_val, ohe_slotwidth_val, ohe_slotheight_val, ohe_advertiser_val, ohe_city_val, \
                             ohe_adexchange_val, ohe_slotformat_val, ohe_slotvisibility_val, ohe_useragent_val], axis=1, join='inner')
df_OHE_val_feat.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  0,  1,  2,
            ...
            29, 30, 31, 32, 33, 34, 35, 36, 37, 38],
           dtype='int64', length=504)

In [22]:
LR_OHE_array_x_all_val= df_OHE_val_feat.as_matrix()
array_y = df_validation[['click']].as_matrix()
array_y_r = np.reshape(array_y, (-1))

In [23]:
# AUC - 0.75 our current best, paper 0.85
def AUC_accuracy(XX_model, array_x, array_y):
    fpr, tpr, thresholds = metrics.roc_curve(array_y, XX_model.predict_proba(array_x)[:, 1])
    return metrics.auc(fpr, tpr)

#print(AUC_accuracy(NB_model_all, array_x_all, array_y_r))
#print(AUC_accuracy(NB_model_3, array_x_3, array_y_r))
#print(AUC_accuracy(LR_model_all, array_x_all, array_y_r))

print(AUC_accuracy(AB_model_all, LR_OHE_array_x_all_val, array_y_r))

0.663550687186


In [37]:
array_pCTRs= AB_model_all.predict_proba(LR_OHE_array_x_all_val)

In [62]:
df_pCTR= pd.DataFrame(array_pCTRs[:, 1])

In [67]:
df_pCTR_click= pd.DataFrame
df_pCTR_click= pd.concat([df_validation[['click', 'bidid']], df_pCTR])

/Users/jamesshields/anaconda/lib/python3.5/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  result = result.union(other)


In [57]:
a = pd.DataFrame(array_pCTRs[:, 1])
a.columns
a.sort_values(0, ascending=[False])

,0
44358,0.487883
187362,0.486808
256860,0.486808
261496,0.486068
114567,0.486068
129651,0.485622
247063,0.485425
189865,0.485180
193953,0.485180
63467,0.485180


In [ ]:
def count_clicks_order(df_pCTRs):
    counter= 0
    dic= {}
    for i in range(len(df_pCTRs)):
        if df_pCTRs[i:(i+1)]['click'].values== 1:
            counter+= 1
            dic[counter]= i
            if counter== 226:
                break
    return dic, counter

dic1= count_clicks_order(df_pCTRs)

## Linear bidding strategy

In [ ]:
df_int2= pd.DataFrame()
df_int2= df_validation
df_int2['pCTR']= NB_model_allpCTR_v
df_int2['pCTR_norm'] = df_int2['pCTR'].values/ df_int2['pCTR'].mean()

In [ ]:
def linear_bid(baseline_bid, imp_pCTR_norm):
    lin_bid = baseline_bid* imp_pCTR_norm
    return lin_bid

def nlinear_bid(baseline_bid, imp_pCTR_norm):
    lin_bid = baseline_bid* (imp_pCTR_norm**2)
    return nlin_bid

def bid_run(df_data_pCTR, baseline_bid, bid_strat= linear_bid, test_run= False, save_run= False):
    
    if test_run== True:    
        df_data_pCTR['imp_bid']= linear_bid(baseline_bid, df_data_pCTR['pCTR_norm'].values)
        
        df_output = pd.DataFrame()
        df_output[['bidid', 'bidprice']]= df_data_pCTR[['bidid', 'imp_bid']]
        
        if save_run== True:
            output_directory= 'Submission/Val/'
            output_filename= 'Group_XX.csv'
            df_output.to_csv(output_directory + output_filename)
            print('Submission file saved: ', os.getcwd(), output_directory, output_filename)
        if save_run== False:
            print('Submission file not saved')
        
        return df_data_pCTR
    
    if test_run== False:
        df_data_pCTR['imp_bid']= linear_bid(baseline_bid, df_data_pCTR['pCTR_norm'].values)
        df_data_pCTR['imp_h_bid']= np.where(df_data_pCTR['imp_bid'] >= df_data_pCTR['bidprice'], 'Y', 'N')
        
        total_cost = 0
        budget = 25000000
        
        df_data_pCTR['imp_win']= 'TBC'
        
        for index, row in df_data_pCTR.iterrows():
            if row['imp_h_bid']== 'Y':
                if (total_cost+ row['bidprice']) < budget:
                    df_data_pCTR.set_value(index,'imp_win', 'Y')
                    total_cost+= row['bidprice']
                else:
                    df_data_pCTR.set_value(index,'imp_win', 'Insuf/b')
            else:
                df_data_pCTR.set_value(index,'imp_win', 'N')
        
        return df_data_pCTR

In [ ]:
df_data_pCTR_bid = pd.DataFrame()
df_data_pCTR_bid = bid_run(df_int2, 1000, test_run= False)

In [ ]:
budget= 25000000

def CTR_evaluation(df_data_pCTR_bid):
    clicks = df_data_pCTR_bid['click'][(df_data_pCTR_bid['click'] == 1)].count()
    total = df_data_pCTR_bid['click'][(df_data_pCTR_bid['click'] != 10)].count()
    total_CTR = clicks/ total
    print('Sample average', total_CTR)
    
    win = df_data_pCTR_bid['click'][(df_data_pCTR_bid['imp_h_bid'] == 'Y')].count()
    win_click = df_data_pCTR_bid['click'][(df_data_pCTR_bid['imp_h_bid'] == 'Y') & (df_data_pCTR_bid.click == 1)].count()
    won_CTR = win_click/ win
    print('Win bids', won_CTR)
    
    loss = df_data_pCTR_bid['click'][(df_data_pCTR_bid['imp_h_bid'] == 'N')].count()
    loss_click = df_data_pCTR_bid['click'][(df_data_pCTR_bid['imp_h_bid'] == 'N') & (df_data_pCTR_bid.click == 1)].count()
    loss_CTR = loss_click/ loss
    print('Not won bids', loss_CTR)
    print('Won: Average ratio', won_CTR/ total_CTR)
    
    print('KPIs---')
    print('Number of clicks bid (NCB)', win_click)
    print('Total spend', df_data_pCTR_bid['bidprice'][(df_data_pCTR_bid['imp_h_bid'] == 'Y')].sum())
    print('Total spend (% budget)', (df_data_pCTR_bid['bidprice'][(df_data_pCTR_bid['imp_h_bid'] == 'Y')].sum()/ budget))
    print('Average CPM (Cost Per Mille)', '???')
    print('Average CPC (Cost Per Click)', (df_data_pCTR_bid['bidprice'][(df_data_pCTR_bid['imp_h_bid'] == 'Y')].sum())/ win_click)
    
CTR_evaluation(df_data_pCTR_bid)

In [ ]:
# Find other evaluation metric
# Optimise for baseline_bid
# Optimise for other pCRT